In [ ]:
import openai
import os
import pickle
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import json
import hashlib

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def generate_assesment(df, scale, prompt_template_fname, llm):

    """
    Given a skill assess its importance to a job
    """

    response_schemas = []
    
    question_schema =  ResponseSchema(name=f"""Assesment""",
                                          description=f"""Assesment of the importance of skill to the job
                                          on a scale of 1 to {scale}"""
                                         )
    response_schemas.append(question_schema)
    answer_schema =  ResponseSchema(name=f"""Reasoning""",
                                          description=f"""Reasoning to support the assesment"""
                                         )
    response_schemas.append(answer_schema)

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()


    prompt_path = os.path.join(os.getenv("PROMPTS"),prompt_template_fname)
    with open(prompt_path,"r") as file:
        template_string = file.read()
    prompt_template = ChatPromptTemplate.from_template(template_string)


    for idx, row in tqdm(df.iterrows()):
        
        skill=row["skill"]
        job=row["job"]
        print(f"""assesing the importance of skill {skill} for job {job}""")

        message = prompt_template.format_messages(
                    skill=skill,
                    job=job,
                    scale=scale,        
                    format_instructions=format_instructions)

        response = llm(message)
        output_dict = output_parser.parse(response.content)
        output_dict["job"]=row["job"]


        path = os.path.join(os.getenv("ASSESMENTS"),f"""assesment_{idx+1}.json""")
        with open(path, "w",  encoding='utf-8') as file:
            json.dump(output_dict,   file, ensure_ascii=False)

## Generate skill assesments

In [ ]:
model = os.getenv("LLM_URL")
#model = "gpt-3.5-turbo"
#model = "gpt-4-turbo-preview"
#llm = ChatOpenAI(temperature=temperature, model=accepted_answer_model)
scale = 10
temperature = 0.2
max_tokens=512
random_seed = 1
prompt_template_fname = "assess_skill_for_job.txt"

llm = ChatOpenAI(
        base_url=os.getenv("LLM_URL"),
        api_key="not-needed",
        temperature=temperature,
        max_tokens=max_tokens,
        model_kwargs={"seed": random_seed}
        )
df = pd.DataFrame()
df["skill"]=["C++","Influencing","Machine Learning","Communication","Leadership"]*2
df["job"]=["Data Scientist","Data Scientist","Data Scientist","Data Scientist","Data Scientist",
           "Product Manager","Product Manager","Product Manager","Product Manager","Product Manager"]
generate_assesment(df, scale, prompt_template_fname, llm)



In [ ]:
[1,2]*2